# Twitter API Interaction
This notebook will walk you through how we will be interacting with the Twitter API to get cashtag-based tweets appropriately. In order to be able to leverage this notebook, you will need to obtain your own Twitter API keys, which you can get by interacting with the [Twitter Developer Portal](https://developer.twitter.com/content/developer-twitter/en).

## Project Setup
In this first brief section, we'll import the required Python libraries and also load in the Twitter API keys from a sensitive file that is intentionally not pushed up to GitHub. (I use a `.gitignore` file to store these keys in a YAML file in a directory called `keys/` that the `.gitignore` file intentionally does not push to GitHub.)

In [7]:
# Importing the necessary Python libraries
import os
import json
import yaml
import requests

In [8]:
# Loading in my personal Twitter API keys from a sensitive file
with open('../keys/twitter-api-keys.yaml', 'r') as f:
    twitter_keys = yaml.safe_load(f)

In [9]:
# Extracting bearer token from the list of all my Twitter API keys
bearer_token = twitter_keys['dkhundley_twitter_creds']['bearer_token']

## Using the Twitter API
While there are Python based wrappers like Tweepy, we are going to be interacting with the Twitter API directly through the `requests` library. This is because the Twitter Python wrappers aren't exactly up-to-date with the latest and greatest stuff, and the functionality we want to use was actually introduced very recently (July 2022).

This notebook will cover everything that we need for this project, but if you specifically want to see which documentation I leveraged for this project, check out these links:

* [General Twitter Developer Portal](https://developer.twitter.com/content/developer-twitter/en)
* [Press Release on New Operators for Twitter API v2](https://developer.twitter.com/en/blog/product-news/2022/twitter-api-v2-filtered-stream)
* [Twitter Introduction to Filtered Streams](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction)
* [Twitter How to Use Filtered Streams](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/api-reference/post-tweets-search-stream-rules)

In [10]:
# Creating a function to handle the OAuth headers
def bearer_oauth(r):
    r.headers['Authorization'] = f'Bearer {bearer_token}'
    r.headers['User-Agent'] = 'v2FilteredStreamPython'
    return r

In [11]:
# Setting the proper API URL
twitter_api_url = 'https://api.twitter.com/2/tweets/search/stream'

### Getting the Rules

In [12]:
# Checking with the Twitter API to see which rules are currently set for the filtered stream
response = requests.get(f'{twitter_api_url}/rules', auth = bearer_oauth)

# Printing out the appropriate response, whether it be the appropriate rules or an error
if response.status_code != 200:
    raise Exception(f'Cannot get rules (HTTP {response.status_code}): {response.text}')
print('Current rules: ')
print(json.dumps(response.json()))

# Saving results to variable
current_rules = response.json()

Current rules: 
{"data": [{"id": "1568051892214812673", "value": "$aapl OR $tsla OR $twtr"}], "meta": {"sent": "2022-09-23T15:40:39.678Z", "result_count": 1}}


### Deleting Old Rules

In [ ]:
# Iterating through any currently existing rules to delete them
if current_rules is not None or 'data' in current_rules:
    
    # Collecting the IDs of the currently existing rules
    current_rule_ids = list(map(lambda rule: rule['id'], current_rules['data']))
    
    # Structuring a delete payload to send to API
    delete_payload = {'delete': {'ids': current_rule_ids}}
    
    # Sending the delete payload to the API
    response = requests.post(f'{twitter_api_url}/rules', auth = bearer_oauth, json = delete_payload)
    
    # Verifying delete results
    if response.status_code != 200:
        raise Exception(f'Cannot delete rules (HTTP {response.status_code}): {response.text}')
    print(json.dumps(response.json()))

### Setting New Rules

In [ ]:
# Setting the query paramaters for our API search results call
new_rules = [
    {'value': '$tsla OR $aapl OR $twtr'}
]

# Structuring payload to add new rules
add_payload = {
    'add': new_rules
}

In [ ]:
# Sending the add payload to the API
response = requests.post(f'{twitter_api_url}/rules', auth = bearer_oauth, json = add_payload)

# Printing out the results appropriately
if response.status_code != 201:
    raise Exception(f'Cannot add rules (HTTP {response.status_code}): {response.text}')
print(json.dumps(response.json()))

### Streaming Twitter Results

In [ ]:
response = requests.get(twitter_api_url, auth = bearer_oauth)
print(response.status_code)
for response_line in response.iter_lines():
    if response_line:
        json_response = json.loads(response_line)
        print(json.dumps(json_response, indent = 4, sort_keys = True))